# <div align="center" style="color: #ff5733;">TSA Closure Third Notification</div>

In [1]:
# Import Libraries   

import pandas as pd  
import numpy as np  

from google.cloud import bigquery
import os

path = r"C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = path
client = bigquery.Client(project="prj-prod-dataplatform")

# Ask Balasubramani to refresh risk_credit_mis.tsa_account_closure table.
**Check if that is refreshed when Balasubramani confirms it**

In [2]:
sq = """select Run_Date, count(1)cnt from risk_credit_mis.tsa_account_closure group by 1 order by 1 desc"""
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
df.head(1)

Job ID 21a89c53-b0c8-4a05-bfdb-79a93396f06f successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,Run_Date,cnt
0,2024-11-21,742455


The date should show yesterday's date

# Read the data from the first sheet for Type1 and Type2 data provided by Marvin

In [3]:
d1 = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\November2024\ThirdNotification_files\Type 1  & 2Nov24.xlsx")
d1.shape

(53501, 2)

In [4]:
print(f"The unique count of customer_id is:\t{len(d1['Customer_Id'].unique())}")
d1.head()


The unique count of customer_id is:	53501


,Customer_Id,ACTION_TYPE
0,1386494,NO ACTION - Type 1
1,1444785,NO ACTION - Type 1
2,1566773,NO ACTION - Type 1
3,1446622,NO ACTION - Type 1
4,1334335,NO ACTION - Type 1


# Read the data from the second sheet for Type3 data provided by Marvin

In [5]:
d2 = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\November2024\ThirdNotification_files\Type 3Nov24.xlsx")
d2.shape

(1, 2)

In [6]:
print(f"The unique count of customer_id is:\t{len(d2['Customer_Id'].unique())}")
d2.head()

The unique count of customer_id is:	1


,Customer_Id,ACTION_TYPE
0,1059331,TYPE 3


In [7]:
result = pd.concat([d1, d2], axis=0, ignore_index=True)
result.shape

(53502, 2)

In [8]:
result.ACTION_TYPE.value_counts()

ACTION_TYPE
NO ACTION - Type 1    53482
TYPE 1                   18
TYPE 2                    1
TYPE 3                    1
Name: count, dtype: int64

In [9]:
result[result['Customer_Id'] == '1059331']

,Customer_Id,ACTION_TYPE


In [10]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53502 entries, 0 to 53501
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_Id  53502 non-null  int64 
 1   ACTION_TYPE  53502 non-null  object
dtypes: int64(1), object(1)
memory usage: 836.1+ KB


In [11]:
result['Customer_Id'] = result['Customer_Id'].astype(str)
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53502 entries, 0 to 53501
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_Id  53502 non-null  object
 1   ACTION_TYPE  53502 non-null  object
dtypes: object(2)
memory usage: 836.1+ KB


In [12]:
result.to_csv("temp.csv", index = False)

In [13]:
a = tuple(result['Customer_Id'])
a[:5]

('1386494', '1444785', '1566773', '1446622', '1334335')

In [14]:
d = '2024-11-21'   ### Need to change this to D-1 everytime we run this code ###
# Get the current date
current_date = pd.Timestamp.now().date()
d_minus_1_days = current_date - pd.DateOffset(days=1)
# Calculate D-60 days
d_minus_60_days = '2024-09-15'
# Calculate D-180 days
d_minus_180_days = '2024-05-18'

print("D-1 day from current date", d_minus_1_days )
print("D-60 day from the current date - 1:", d_minus_60_days)
print("D-180 day from the current date -1:", d_minus_180_days)

d_minus_1_days_string = d_minus_1_days.strftime('%Y-%m-%d')
print("D-1 day from the current date:", d_minus_1_days_string)

d_minus_60_days_string = d_minus_60_days
print("D-60 day from the current date:", d_minus_60_days_string)

d_minus_180_days_string = d_minus_180_days
print("D-180 day from the current date:", d_minus_180_days_string)

D-1 day from current date 2024-11-21 00:00:00
D-60 day from the current date - 1: 2024-09-15
D-180 day from the current date -1: 2024-05-18
D-1 day from the current date: 2024-11-21
D-60 day from the current date: 2024-09-15
D-180 day from the current date: 2024-05-18


Step1: At line 11 change the date to D-1<br><br>
Step2: At line around 36 (and (TSA_Opening_date < date '2023-10-18')   ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step3: At line around 45 (WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null and (TSA_Opening_date < date '2023-10-18')  ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step4: At line 58 (and TSA_Opening_date < date '2023-06-18'  and (Last_TSA_Credit_Date < date '2023-06-18' and Last_TSA_Debit_Date < date '2023-06-18' and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01') < date '2023-06-18')  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) <br><br>
Step5: At line 72 (Run_Date = date '2024-01-09'###DATE change here to D-1) Change the date to D-1<br><br>

First Notification we change all the steps <br><br>
Second and Third Notification we only change Step1 and Step5
For Second Notificaiton and Third Notificaiton the customre id should be from the file sent by Marvin. <br><br>

In [15]:
sq = f"""
with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '{d_minus_1_days_string}' ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  --customer_id in (select cast(customer_id as string) from prj-prod-dataplatform.dap_ds_poweruser_playground.tsa_2nd_notification_base_10jan2024)
  customer_id in {a}
)
select * from w_tsa;
"""

dfd = client.query(sq).to_dataframe(progress_bar_type =  'tqdm')
dfd.shape

Job ID ba7a6abd-445b-417a-8a5a-b09d34373dac successfully executed: 100%|██████████|
Downloading: 100%|██████████|


(53501, 29)

In [16]:
dfd.head()

,Customer_Id,TSA_Opening_date,TSA_Status,TSA_Balance,Solo_Stash_Balance,Group_Stash_Balance,Term_Deposit_Balance,Active_Term_Deposit_flag,Last_TSA_Credit_Date,Last_TSA_Debit_Date,...,Last_TSA_EOD_Balance_Date,Last_Group_Stash_Credit_Date,Group_Stash_Balance_Owner,Run_Date,Batch,Processing_Month,pending_loans_not_expired_not_exempt,expired_loans,pending_loans_lastDt,expired_loans_lastDt
0,1082292,2021-09-02,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,...,NaT,NaT,NaN,2024-11-21,Batch_1,November,0,0,NaT,NaT
1,1510791,2022-07-31,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,...,NaT,NaT,NaN,2024-11-21,Batch_1,November,0,0,NaT,NaT
2,1419444,2022-06-22,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,...,NaT,NaT,NaN,2024-11-21,Batch_1,November,0,0,NaT,NaT
3,1396533,2022-06-03,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,...,NaT,NaT,NaN,2024-11-21,Batch_1,November,0,0,NaT,NaT
4,1074567,2021-08-08,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,...,NaT,NaT,NaN,2024-11-21,Batch_1,November,0,0,NaT,NaT


In [17]:
sq = f"""
with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '{d_minus_1_days_string}'  ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  --customer_id in (select cast(customer_id as string) from prj-prod-dataplatform.dap_ds_poweruser_playground.tsa_2nd_notification_base_10jan2024)
  customer_id in {a}
),
w_approved
as
(
  select DISTINCT customerId from risk_credit_mis.loan_master_table where loantype = 'BIGLOAN' OR FLAGAPPROVAL = 1 OR FLAGDISBURSEMENT = 1  
  or (stage1Status = 'EXPIRED' AND cddInitiatedDateAndTime IS NULL)
)
select *
from
(
  select distinct
  a.Customer_Id,a.TSA_Opening_date,
  c.OFSTANDARDACCOUNTID as TSA_Account_Number, c.OfblockAmount, 
  a.TSA_Status,a.TSA_Balance,a.Solo_Stash_Balance,a.Group_Stash_Balance,a.Term_Deposit_Balance,a.Active_Term_Deposit_flag,a.Last_TSA_Credit_Date,a.Last_TSA_Debit_Date,
  a.ARI_Flag,a.Active_Loan,a.Total_Loan_Outstanding,  case when b.customerId is not null then 1 else 0 end as Approved_Loans,
  a.Pending_Loan_Applications,a.Hard_Reject_Loans_App,a.Soft_Reject_Loans_App,a.Cancelled_Loans_App, extract(date from a.Last_Login_Date) as Last_Login_Date,
  a.Last_TSA_EOD_Balance_Date,a.Last_Group_Stash_Credit_Date,a.Group_Stash_Balance_Owner,a.Run_Date,a.Batch,a.Processing_Month,a.pending_loans_not_expired_not_exempt,a.expired_loans,
  a.pending_loans_lastDt,a.expired_loans_lastDt,
  CASE 
    WHEN tsa_status = 'Active' and (Last_TSA_Credit_Date = date '1970-01-01' and Last_Group_Stash_Credit_Date is null)
    and (TSA_Opening_date < date '{d_minus_60_days_string}')   ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (coalesce(Active_Loan,0) = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)  THEN 'TYPE 1'
    and b.customerId is null  THEN 'TYPE 1'


    WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null and (TSA_Opening_date < date '{d_minus_60_days_string}')  ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (coalesce(Active_Loan, 0) = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
    and b.customerId is null  
    and (date_trunc(TSA_Opening_date,day) = date_trunc(Last_Login_Date,day)) THEN 'TYPE 2'

    
    WHEN ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) < 1 )
            and (coalesce(Active_Loan, 0) = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
            and (Active_Term_Deposit_flag = 0)
            and TSA_Opening_date < date '{d_minus_180_days_string}'                                     ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and (Last_TSA_Credit_Date < date '{d_minus_180_days_string}'                                 ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and Last_TSA_Debit_Date < date '{d_minus_180_days_string}'                                         ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01') < date '{d_minus_180_days_string}' )  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and tsa_status = 'Active'
            and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
            and b.customerId is null  
    THEN 'TYPE 3'
    ELSE 'NO ACTION'
    END AS ACTION_TYPE
  from w_tsa A
  left join w_approved b
  on cast(a.Customer_Id as int64) = b.customerId
  left join core_raw.customer_accounts c
  on c.OFCUSTOMERID = a.Customer_Id and c.CRINTERDESC = 'Transactional Savings Account Inv_R' and c.OFISCLOSED = 'N'
  where
  Run_Date = date '{d_minus_1_days_string}'  ###DATE change here to D-1
)
"""


In [18]:
print(sq)


with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '2024-11-21'  ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  --customer_id in (select cast(customer_id as string) from prj-prod-dataplatform.dap_ds_poweruser_playground.tsa_2nd_notification_base_10jan2024)
  customer_id in ('1386494', '1444785', '1566773', '1446622', '1334335', '1538847', '1516331', '1526659', '1157901', '1282593', '1295389', '1439584', '1500590', '1608021', '1572412', '1560039', '1303342', '1326211', '1143998', '1579944', '1272570', '1278431', '1586137', '1374021', '1352904', '1192539', '1173763', '1334275', '1470512', '1489418', '1492754', '1406390', '1503836', '1512346', '1535486', '1452767', '1520752', '1491915',

In [19]:
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 981302ee-c07c-4a9a-93c9-d0ff16dc399c successfully executed: |          |
Downloading: 100%|██████████|


In [20]:
pd.set_option('display.max_columns', None)
df.sample(10)

,Customer_Id,TSA_Opening_date,TSA_Account_Number,OfblockAmount,TSA_Status,TSA_Balance,Solo_Stash_Balance,Group_Stash_Balance,Term_Deposit_Balance,Active_Term_Deposit_flag,Last_TSA_Credit_Date,Last_TSA_Debit_Date,ARI_Flag,Active_Loan,Total_Loan_Outstanding,Approved_Loans,Pending_Loan_Applications,Hard_Reject_Loans_App,Soft_Reject_Loans_App,Cancelled_Loans_App,Last_Login_Date,Last_TSA_EOD_Balance_Date,Last_Group_Stash_Credit_Date,Group_Stash_Balance_Owner,Run_Date,Batch,Processing_Month,pending_loans_not_expired_not_exempt,expired_loans,pending_loans_lastDt,expired_loans_lastDt,ACTION_TYPE
35837,1561257,2022-08-22,60815612570008,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,1,0,0,1,2022-08-22,NaT,NaT,NaN,2024-11-21,Batch_1,November,1,0,2022-08-22,NaT,NO ACTION
38437,1446782,2022-07-06,60814467820008,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,1,0,0,0,2022-07-06,NaT,NaT,NaN,2024-11-21,Batch_1,November,1,0,2022-08-11,NaT,NO ACTION
2853,1606003,2022-09-08,60816060030004,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,1,0,0,0,2022-09-08,NaT,NaT,NaN,2024-11-21,Batch_1,November,1,0,2022-10-09,NaT,NO ACTION
34250,1452852,2022-07-09,60814528520004,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,1,0,0,0,2022-07-09,NaT,NaT,NaN,2024-11-21,Batch_1,November,1,0,2022-08-11,NaT,NO ACTION
32226,1480850,2022-07-21,60814808500009,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,1,0,0,0,2022-07-30,NaT,NaT,NaN,2024-11-21,Batch_1,November,1,0,2022-08-22,NaT,NO ACTION
49542,1459489,2022-07-11,60814594890003,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,1,0,1,0,2024-09-04,NaT,NaT,NaN,2024-11-21,Batch_1,November,1,0,2022-08-11,NaT,NO ACTION
6802,1296476,2022-04-02,60812964760004,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,1,0,0,1,2023-03-28,NaT,NaT,NaN,2024-11-21,Batch_1,November,1,2,2022-09-25,2023-05-26,NO ACTION
3976,1549996,2022-08-16,60815499960008,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,1,0,0,0,2024-05-18,NaT,NaT,NaN,2024-11-21,Batch_1,November,0,1,NaT,2024-08-17,NO ACTION
50530,1539904,2022-08-11,60815399040003,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,2,0,0,0,2022-11-03,NaT,NaT,NaN,2024-11-21,Batch_1,November,1,0,2022-09-11,NaT,NO ACTION
42783,1552288,2022-08-17,60815522880004,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,2,0,0,2,2022-11-16,NaT,NaT,NaN,2024-11-21,Batch_1,November,2,1,2022-08-18,2022-11-17,NO ACTION


In [21]:
df.shape

(53501, 32)

In [22]:
df['ACTION_TYPE'].value_counts()

ACTION_TYPE
NO ACTION    53479
TYPE 1          20
TYPE 3           1
TYPE 2           1
Name: count, dtype: int64

In [23]:
df.to_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\November2024\ThirdNotification_files\20241121_TSA_Account_Closure_3rdtNotification.xlsx", sheet_name='TSA_Closure_3rdNotification', index = False)

In [24]:
df = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\November2024\ThirdNotification_files\20241121_TSA_Account_Closure_3rdtNotification.xlsx", sheet_name='TSA_Closure_3rdNotification')
df.shape

(53501, 32)

In [25]:
df.columns

Index(['Customer_Id', 'TSA_Opening_date', 'TSA_Account_Number',
       'OfblockAmount', 'TSA_Status', 'TSA_Balance', 'Solo_Stash_Balance',
       'Group_Stash_Balance', 'Term_Deposit_Balance',
       'Active_Term_Deposit_flag', 'Last_TSA_Credit_Date',
       'Last_TSA_Debit_Date', 'ARI_Flag', 'Active_Loan',
       'Total_Loan_Outstanding', 'Approved_Loans', 'Pending_Loan_Applications',
       'Hard_Reject_Loans_App', 'Soft_Reject_Loans_App', 'Cancelled_Loans_App',
       'Last_Login_Date', 'Last_TSA_EOD_Balance_Date',
       'Last_Group_Stash_Credit_Date', 'Group_Stash_Balance_Owner', 'Run_Date',
       'Batch', 'Processing_Month', 'pending_loans_not_expired_not_exempt',
       'expired_loans', 'pending_loans_lastDt', 'expired_loans_lastDt',
       'ACTION_TYPE'],
      dtype='object')

In [26]:
df[df['ACTION_TYPE']!='NO ACTION'].groupby('ACTION_TYPE')['Customer_Id'].nunique()

ACTION_TYPE
TYPE 1    20
TYPE 2     1
TYPE 3     1
Name: Customer_Id, dtype: int64

In [27]:
25780+69+859

26708

In [28]:
df.groupby('ACTION_TYPE')['Customer_Id'].count()

ACTION_TYPE
NO ACTION    53479
TYPE 1          20
TYPE 2           1
TYPE 3           1
Name: Customer_Id, dtype: int64

In [29]:
25781+69+859+162

26871

In [30]:
len(df['Customer_Id'].unique())

53501